## **Public API**
Use public API to create a data frame

Yu-Gi-Oh! API guide: https://ygoprodeck.com/api-guide/

Yu-Gi-Oh! TCG Information API: https://db.ygoprodeck.com/api/v7/cardinfo.php

### **Import libraries**

In [ ]:
import requests
import time            
import pandas as pd
import re   # regular expression

### **Test connection**

In [ ]:
# test
test_url = "https://db.ygoprodeck.com/api/v7/cardinfo.php?&type=Effect%20Monster&fname=Stardus"

test_response = requests.get(test_url)

print(f"status code: {test_response.status_code}")

test_result = test_response.json()

# test_result['data'][1]
# print(test_result['data'][1]['name'])
# print(test_result['data'][1]['type'])
# print(test_result['data'][1]['archetype'])

status code: 200


### **Prepare empty lists for creating a data frame**

In [ ]:
# The Card Information endpoint -> https://db.ygoprodeck.com/api/v7/cardinfo.php

# selected cards
card_type = ['Effect%20Monster', 'Tuner%20Monster','Synchro%20Monster', 'Spell%20Card', 'Trap%20Card']
card_name = ['Junk', 'Stardus']

# prepared lists for a data frame
ids = []
names = []
types = []
descs = []
atks = []
defs = []
levels = []
races = []
attributes = []
archetypes = []

### **Request data & create a data frame**

In [ ]:
# requests.get() loop
for cname in card_name:         # loop Junk and Stardus
    for ctype in card_type:     # loop card type
        url = f"https://db.ygoprodeck.com/api/v7/cardinfo.php?&type={ctype}&fname={cname}"
        response = requests.get(url)
        results = response.json()
        monster = re.findall('[A-z]+[%][0-9]+Monster$', ctype)      # chack if it's a monster card
        if monster == []:
            monster.append('Spell or Trap')                         # if it's not a monster card -> append 'Spell or Trap' instead
        for result in results['data']:
            if 'archetype' in result.keys():                        # some cards don't have archetypes
                if monster[0] in card_type:                         # monster cards have these 4 parameters
                    atks.append(result['atk'])
                    defs.append(result['def'])
                    levels.append(result['level'])
                    attributes.append(result['attribute'])
                else:                                               # spell & trap cards don't have these 4 parameters
                    atks.append('-')
                    defs.append('-')
                    levels.append('-')
                    attributes.append('-')
                ids.append(result['id'])
                names.append(result['name'])
                types.append(result['type'])
                descs.append(result['desc'])
                races.append(result['race'])
                archetypes.append(result['archetype'])
                
                time.sleep(3)                                       # sleep for 3 seconds

# create a date frame
my_deck = pd.DataFrame({
    'id': ids,
    'name': names,
    'type': types,
    'desc': descs,
    'atk': atks,
    'def': defs,
    'level': levels,
    'race': races,
    'attribute': attributes,
    'archetype': archetypes
})

### **Queries**

In [ ]:
my_deck.head()

,id,name,type,desc,atk,def,level,race,attribute,archetype
0,25148255,Junk Anchor,Tuner Monster,"For a Synchro Summon, you can substitute this ...",0,0,2,Warrior,EARTH,Junk
1,51855378,Junk Blader,Effect Monster,"You can banish 1 ""Junk"" monster from your Grav...",1800,1000,4,Warrior,EARTH,Junk
2,43436049,Junk Breaker,Effect Monster,"During your Main Phase, if this card was Norma...",1800,1000,4,Warrior,EARTH,Junk
3,1006081,Junk Changer,Tuner Monster,"You can only use the following effect of ""Junk...",1500,900,3,Warrior,EARTH,Junk
4,58242947,Junk Collector,Effect Monster,(Quick Effect): You can banish this card on th...,1000,2200,5,Warrior,LIGHT,Junk


In [ ]:
my_deck.query("type == 'Synchro Monster' & race == 'Dragon'")

,id,name,type,desc,atk,def,level,race,attribute,archetype
34,30983281,Accel Synchro Stardust Dragon,Synchro Monster,1 Tuner + 1+ non-Tuner monsters\r\nIf this car...,2500,2000,8,Dragon,WIND,Stardust
37,89474727,Stardust Chronicle Spark Dragon,Synchro Monster,1 Tuner Synchro Monster + 1+ non-Tuner Synchro...,3000,2500,10,Dragon,LIGHT,Stardust
38,44508094,Stardust Dragon,Synchro Monster,1 Tuner + 1+ non-Tuner monsters\r\nWhen a card...,2500,2000,8,Dragon,WIND,Stardust
39,26268488,Stardust Sifr Divine Dragon,Synchro Monster,1 Tuner Synchro Monster + 2 or more non-Tuner ...,4000,4000,12,Dragon,LIGHT,Stardust
40,83994433,Stardust Spark Dragon,Synchro Monster,1 Tuner + 1+ non-Tuner monsters\nOnce per turn...,2500,2000,8,Dragon,LIGHT,Stardust


In [ ]:
my_deck.query("archetype not in ('Junk', 'Stardust')")

,id,name,type,desc,atk,def,level,race,attribute,archetype
22,67968069,Junk Puppet,Spell Card,"Target 1 ""Gimmick Puppet"" monster in your GY; ...",-,-,-,Normal,-,Gimmick Puppet
25,36521459,Malefic Stardust Dragon,Effect Monster,Cannot be Normal Summoned or Set. Must be Spec...,2500,2000,8,Dragon,DARK,Malefic
26,61257789,Stardust Dragon/Assault Mode,Effect Monster,Cannot be Normal Summoned/Set. Must first be S...,3000,2500,10,Dragon,WIND,Assault Mode
32,95403418,Starduston,Effect Monster,Cannot be Normal Summoned/Set. Must be Special...,0,0,1,Fiend,DARK,Duston
